In [12]:
import numpy as np
import re
import jieba
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import time
%matplotlib inline

In [2]:
csv_path = "C:/study/AI Training and Jobs/NLP_CV_Course/datasource-master/export_sql_1558435/sqlResult_1558435.csv"
stop_words_path = "C:/study/AI Training and Jobs/NLP_CV_Course/datasource-master/"
cleaned_data_path = "C:/study/AI Training and Jobs/NLP_CV_Course/week_7/"

In [3]:
news = pd.read_csv(csv_path, encoding='gb18030')

In [4]:
news.dropna(subset=["content", "source"], inplace = True)

In [5]:
# with open(cleaned_data_path + 'jieba_data.txt', 'r', encoding='UTF-8') as f:
#     lines = f.readlines()
# news_pos = news[news['source'].str.contains("新华社")]
# news_neg = news[news['source'].str.contains("新华社") == 0]
# news_pos = news_pos.sample(len(news_neg))
# news_sampled = news_pos.append(news_neg)

## Generating Tfidf matrix

In [6]:
with open(cleaned_data_path + 'jieba_data.txt', 'r', encoding='UTF-8') as f:
    lines = f.readlines()
corpus = pd.Series(lines)
print("corpus's shape is ", corpus.shape)
vectorizer = TfidfVectorizer(max_features = 350, stop_words = [])
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print("input of X's shape is ", X.shape)
train_data = X.toarray()
print("train_data's shape is {}".format(train_data.shape))

corpus's shape is  (87052,)
['nba', 'nn', 'nnn', 'nnnum', 'nnum', 'num', 'numnum', '一个', '一些', '一名', '一带', '一次', '一直', '一起', '一路', '万元', '上海', '不仅', '不同', '不少', '不断', '不是', '不能', '专业', '专家', '世界', '业务', '中国', '中心', '为了', '主场', '主席', '主要', '举办', '举行', '之一', '之后', '习近平', '了解', '事件', '二线', '互联网', '交流', '产业', '产品', '人们', '人员', '人民', '人民币', '亿元', '今年', '介绍', '他们', '代表', '以上', '以及', '以来', '价格', '企业', '会议', '传统', '位于', '体育', '作为', '作品', '使用', '促进', '俄罗斯', '保护', '信息', '健康', '全国', '全球', '全面', '公司', '公开赛', '共享', '共同', '关于', '关注', '关系', '其中', '其他', '具有', '内容', '冠军', '决定', '决赛', '出现', '分别', '创新', '利用', '制造', '加强', '包括', '北京', '区域', '历史', '去年', '参与', '参加', '双方', '发展', '发布', '发现', '发生', '取得', '叙利亚', '可以', '可能', '台湾', '合作', '同时', '启动', '品牌', '因为', '国内', '国家', '国际', '图表', '地区', '地方', '城市', '基础', '基金', '增加', '增长', '外代', '大学', '夺冠', '如何', '如果', '媒体', '存在', '学习', '学校', '学生', '孩子', '安全', '完成', '实施', '实现', '宣布', '对于', '就是', '展示', '工作', '工程', '已经', '巴黎', '市场', '希望', '带来', '帮助', '平台', '广州', '庆祝', '建立', '建设',

## Preparing for train and test data

In [7]:
sourceMapping = {}
for index, label in enumerate(set(news['source'])):
    if label == "新华社":
        sourceMapping['新华社'] = 1
    else:
        sourceMapping[label] = 0
news['source'] = news['source'].map(sourceMapping)
test_data = news['source']
print("test_data's shape is {}".format(test_data.shape))
train_input, test_input, train_output, test_output = train_test_split(train_data, test_data, test_size=0.2, random_state=42)
x_arr_train = train_input
x_arr_test = test_input
y_arr_train = train_output.as_matrix()
y_arr_test = test_output.as_matrix()

test_data's shape is (87052,)


C:\Users\WDMWDL\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
C:\Users\WDMWDL\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


#### performance evaluation 

In [8]:
def performance_evaluation(clf):
#     print("model's score is {}".format(clf.score(x_arr_test, y_arr_test)))
    y_arr_test_pred = clf.predict(x_arr_test)
    print("precision is {}".format(precision_score(y_arr_test, y_arr_test_pred)))
    print("recall score is {}".format(recall_score(y_arr_test, y_arr_test_pred)))
    print("f1 is {}".format(f1_score(y_arr_test, y_arr_test_pred)))

### Model construction with Bayes

In [9]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_arr_train[1:45000,:], y_arr_train[1:45000])
performance_evaluation(gnb)

precision is 0.9943319226118501
recall score is 0.835046966235085
f1 is 0.9077549330757555


### Model construction with random forest

In [10]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1,max_depth=30, random_state=0)
clf.fit(x_arr_train, y_arr_train)
performance_evaluation(clf)

precision is 0.9911919396742919
recall score is 0.9927646610814927
f1 is 0.9919776770143006


### Model construction with SVM

In [ ]:
from sklearn.svm import SVC
start = time.clock()
clf = SVC(gamma='auto', random_state=42, max_iter=500, kernel ='linear')
clf.fit(x_arr_train[1:10000], y_arr_train[1:10000])
elapse = time.clock() - start
print("consuming time is {}".format(elapse))
performance_evaluation(clf)

C:\Users\WDMWDL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
C:\Users\WDMWDL\Anaconda3\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\WDMWDL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """


consuming time is 15.594910499000036
